In [ ]:
# Run in python 2.7+
# Script for all sensor data

import arable
from   arable.client   import ArableClient
import re
from   datetime        import datetime, timedelta
import pandas          as     pd
from   io              import StringIO
from   sensor_includes import email, password_ccber, tenant_ccber

In [ ]:
def change(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
def changeback(x):
    return datetime.strptime(x, "%Y-%m-%d")
def tests(x):
    return x.strftime("%Y-%m")
def ha(a, b):
    a[0].append(change(b))
    a[1].append(tests(b))
    a[2].append(str(b))
    a[3].append(tests(b))
def haha(a, b):
    a[0].append(str(b))
    a[1].append(str(tests(b)))
def hahaha(a, b):
    for i in range(len(a)):
        a[i].append(b[i])
def hahahaha(a, b):
    a[2].append(str(b))
    a[3].append(tests(b))

In [ ]:
a = ArableClient()
a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

In [ ]:
device_ids     = [] # Ex. A00****
device_names   = [] # Ex. Lagoon Ice Plants
device_created = [] # Ex. 2019-10-10T00:00:00

# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_names.
# The corresponding Device ID is added to device_ids.
for i in a.devices():
    for j in range(len(i.keys())):
        if str(i.keys()[j]).startswith('location'):               
            loc = str(i.values()[j])
            loc_name = (re.split(r':', loc)[2])[3:-2]
            if re.search("Z", loc_name):
                device_ids.append(str(i.values()[j-30]))
                device_names.append(loc_name)
                device_created.append(str(i.values()[j-28]))

In [ ]:
# Converts each item in device_ids to a list.          
device_ids = list(map(lambda el:[el], device_ids))

In [ ]:
# This can be deleted later.
s_month = []
s_year  = []
for i in device_created:
    s_month.append(int(i[5:-19]))
    s_year.append(int(i[:-22]))

In [ ]:
dt      = datetime.now()

sta_m   = []
sta_m_f = []
end_m   = []
end_m_f = []
mm      = [sta_m, sta_m_f, end_m, end_m_f]

sta_y   = []
sta_y_f = []
end_y   = []
end_y_f = []
yy      = [sta_y, sta_y_f, end_y, end_y_f]

In [ ]:
for j in range(len(device_created)):
    sta_m_t   = []
    sta_m_f_t = []
    end_m_t   = []
    end_m_f_t = []
    sta_y_t   = []
    sta_y_f_t = []
    end_y_t   = []
    end_y_f_t = []
    mo = [sta_m_t, sta_m_f_t, end_m_t, end_m_f_t]
    ye = [sta_y_t, sta_y_f_t, end_y_t, end_y_f_t]
    s  = changeback((device_created[j])[:10])
    haha(mo, s)
    haha(ye, s)
    s  = s.replace(day=1)
    if s.year == dt.year:
        for i in range(s.month+1, dt.month+1):
            ha(mo, s.replace(month=i))
    elif s.year != dt.year: 
        for i in range(dt.year-s.year):
            for j in range(s.month+1, 13):
                ha(mo, s.replace(month=j))
        for i in range(s.year+1, dt.year+1):
            x = s.replace(year=i, month=1)
            ha(ye, x)
            for j in range(1, dt.month+1):
                ha(mo, x.replace(month=j)) 
    hahahaha(mo, dt)
    hahahaha(ye, dt)
    hahaha(mm, mo)
    hahaha(yy, ye)

In [ ]:
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
def test(sta, end, i, sta_f, end_f, device):
    for j in range(len(sta)):
        try:
            df = a.query(select = 'all', format = 'csv', devices = device, measure = str(i), order = "time", end = end[j], start = sta[j], limit = 100000000)
            fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
            print(fn)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            df = df.drop('location', 1)
            df.to_csv(fn, sep = ",")
        except:
            print('ERROR WRITING FILE')
            continue

In [ ]:
for i in range(len(device_ids)):
    test(sta_m[i], end_m[i], "hourly", sta_m_f[i], end_m_f[i], device_ids[i])
    test(sta_y[i], end_y[i], "daily", sta_y_f[i], end_y_f[i], device_ids[i])